Tokenization can have a significant impact on the quality of models. In this task, you will need to reduce the tokenizer to reduce the size of the model while maintaining quality. You will solve the problem of sentiment analysis. In the first part of the notebook, we implement a baseline, which measures the quality of the original model. In the second part, you need to reduce the tokenizer by removing at least 50% of tokens from it, reinitialize the embedding layer of BERT model and retrain the model. The classification quality should decrease by no more than 2% in terms of the F-measure, compared to the original model.

### Baseline

In [ ]:
!pip install transformers datasets evaluate accelerate tokenizers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    BertTokenizerFast
)
from datasets import load_dataset
import evaluate
import os
from tokenizers import BertWordPieceTokenizer

In [ ]:
# 1. Загрузка данных
dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(seed=42).select(range(5000))
test_data = dataset["test"].shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# 2. Исходный токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# 3. Обучение исходной модели
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

model.cuda()

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to=['tensorboard'],
    bf16=True,
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.395200,0.391385,0.863000
2,0.261100,0.418607,0.890000
3,0.153800,0.513955,0.895000


TrainOutput(global_step=1875, training_loss=0.23743114013671876, metrics={'train_runtime': 1099.2774, 'train_samples_per_second': 13.645, 'train_steps_per_second': 1.706, 'total_flos': 1973332915200000.0, 'train_loss': 0.23743114013671876, 'epoch': 3.0})

In [ ]:
# 4. Оценка исходной модели
original_accuracy = trainer.evaluate()["eval_accuracy"]
print(f"Исходная точность: {original_accuracy:.4f}")

Исходная точность: 0.8950


### Task 1

Remove at least 50% of tokens from original tokenizer. Reinitialize new tokenizer. Give to new tokenizer 'new_tokenizer' name.

In [ ]:
from collections import Counter

def get_tail_k_tokens(tokenizer: AutoTokenizer, texts: list, k: int = 1000) -> list:
    token_counts = Counter()
    batch_size = 1000

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        encodings = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

        for encoding in encodings["input_ids"]:
            tokens = tokenizer.convert_ids_to_tokens(encoding.tolist())
            token_counts.update(tokens)

    tail_k_tokens = [token for token, count in token_counts.most_common()[:-k-1:-1]]

    return tail_k_tokens

In [ ]:
from transformers import BertTokenizer

texts = train_data['text'] + test_data['text']
tokens_to_remove = set(get_tail_k_tokens(tokenizer, texts, k=round(len(tokenizer.vocab) * 0.5)))
tokens_to_remove.update({'.', ',', '!', '?', '#', '&', '$'})
special_tokens = {tokenizer.cls_token, tokenizer.sep_token,
                  tokenizer.pad_token, tokenizer.unk_token, tokenizer.mask_token}
tokens_to_remove = tokens_to_remove - special_tokens

new_token_list = [token for token in tokenizer.get_vocab().keys() if token not in tokens_to_remove]
new_token2id = {token: idx for idx, token in enumerate(new_token_list)}

vocab_file = "new_vocab.txt"
with open(vocab_file, "w", encoding="utf-8") as f:
    for token in new_token_list:
        f.write(token + "\n")

new_tokenizer = BertTokenizer(vocab_file=vocab_file)

print(f"\nРазмеры словарей:")
print(f"Исходный: {len(tokenizer.vocab)}")
print(f"Новый: {len(new_tokenizer.vocab)}")
print(f"Удалено токенов: {len(tokenizer.vocab) - len(new_tokenizer.vocab)}")
print(f"Удалено токенов в %: {(len(tokenizer.vocab) - len(new_tokenizer.vocab)) * 100 / len(tokenizer.vocab)}")


Размеры словарей:
Исходный: 30522
Новый: 15254
Удалено токенов: 15268
Удалено токенов в %: 50.02293427691501


### Task 2:

Initialize new Embedding layer of BERT model according to new tokenizer.
Retrain model on classification task. The classification quality should decrease by no more than 2% in terms of the F-measure, compared to the original model.

In [ ]:
import torch.nn as nn


# 7. Переинициализация модели с новым словарем
new_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    ignore_mismatched_sizes=True  # Важно для изменения размера эмбеддингов!
)

old_embeddings = new_model.get_input_embeddings().weight.data
hidden_size = old_embeddings.shape[1]
new_vocab_size = len(new_token_list)

new_embeddings = torch.zeros((new_vocab_size, hidden_size))
for token, new_idx in new_token2id.items():
    old_idx = tokenizer.get_vocab()[token]
    new_embeddings[new_idx] = old_embeddings[old_idx]

new_embedding_layer = nn.Embedding(new_vocab_size, hidden_size)
new_embedding_layer.weight.data = new_embeddings
new_model.set_input_embeddings(new_embedding_layer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 8. Обучение с новым токенизатором
def new_tokenize_function(examples):
    return new_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

new_train_dataset = train_data.map(new_tokenize_function, batched=True)
new_test_dataset = test_data.map(new_tokenize_function, batched=True)

new_trainer = Trainer(
    model=new_model,
    args=training_args,
    train_dataset=new_train_dataset,
    eval_dataset=new_test_dataset,
    compute_metrics=compute_metrics,
)

new_trainer.train()
new_accuracy = new_trainer.evaluate()["eval_accuracy"]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.399300,0.405742,0.853000
2,0.262500,0.440645,0.884000
3,0.151000,0.535544,0.884000


In [ ]:
# 9. Результаты
print("\nРезультаты сравнения:")
print(f"Исходная точность: {original_accuracy:.4f}")
print(f"Точность с новым словарем: {new_accuracy:.4f}")
print(f"Разница: {original_accuracy - new_accuracy:.4f}")
print(f"Экономия памяти: {(len(tokenizer.vocab) - len(new_tokenizer.vocab))/len(tokenizer.vocab):.2%}")


Результаты сравнения:
Исходная точность: 0.8950
Точность с новым словарем: 0.8840
Разница: 0.0110
Экономия памяти: 50.02%
